<a href="https://colab.research.google.com/github/MelihKlc/Machine-Learning/blob/main/convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator  #for image preprocessing

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, #her pixeli 255 e bölerek feature scaling yapıyor.(compulsary)
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True) #overfitting olmaması için sadece training sette bir transformation yapmamız gerek. Bu transformationlara image augmentation diyoruz.
 training_set = train_datagen.flow_from_directory('dataset/training_set', #training sete bağlıyoruz flow_from_directory ile augment edilmiş image leri
                                                 target_size = (64, 64), #final size of our images
                                                 batch_size = 32, #her batch de kaç image olacak onu belirliyoruz.
                                                 class_mode = 'binary')    #we have binary outcome, cat or dog

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255) #her zaman yaptığımız gibi normalde de test setlere sadece transformation uyguluyoduk fit uygulamıyoduk verileri bozmamak için burda da augment yapmıcaz teste çünkü original fotoların bozulmasını istemiyoruz. Fakat feature scaling yapıyoruz çünkü biliyoruz ki yine önceden verilerin scale i aynı olmalı. Train sette scaleing yaptıgımız icin burda da yapıyoruz.
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()  #sequential layerlar kullanacağımız için onları bununla oluşturuyoruz.

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32 , kernel_size = 3 , activation = 'relu' , input_shape[64,64,3] ))  #add methodunu layer eklemek için kullanıyoruz.filters yani feature detector sayısını belirlicez. kernel size da aslında feature filtersin dağılım şekli eğer 3 seçersek 3x3 matrix oluşuyo. bir de input shape belirlicez onu da üstte 64x64 belirlediğimiz için 64 64 3 giricez 3 te biz renkli resim kullandığımız için RGB olarak 3 belirliyoruz 3 boyut.
#filters genel olarak 32 seçiliyor. activation olarak da rectifier seçiyoruz

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2 , strides = 2)) #max pool kullanıyoruz ve 2x2 seçiyoruz poolsize parametresiyle. strides parametresiyle de kaç pixel atlayıp pooling mapimizi oluşturuyoruz onu seçiyoruz. padding parametresinde valid dersek mesela en sağda 2 pixel giriyo sadece içine poolun sadece o 2 pixele göre işlem yapıyor. eğer same seçersek boş olan 2 pixeli fake pixellerle 0 larla yani dolduruyor.

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32 , kernel_size = 3 , activation = 'relu'  ))  #add methodunu layer eklemek için kullanıyoruz.filters yani feature detector sayısını belirlicez. kernel size da aslında feature filtersin dağılım şekli eğer 3 seçersek 3x3 matrix oluşuyo. bir de input shape belirlicez onu da üstte 64x64 belirlediğimiz için 64 64 3 giricez 3 te biz renkli resim kullandığımız için RGB olarak 3 belirliyoruz 3 boyut.
#filters genel olarak 32 seçiliyor. activation olarak da rectifier seçiyoruz. sadece first conv layer direkt olarak inputa geçtiği için ikincide input shape belirlememize gerek yok
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2 , strides = 2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten()) #bu input oluyor artık bu inputu da full connectiona bağlıcaz.

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units = 128 , activation = 'relu')) #unitsle neuron sayısını seçtik layerin içindeki.

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units = 1 , activation = 'sigmoid'))  #şimdi outputlarımız kedi ve köpek olduğu için sonucunda 1 köpek 0 kedi olarak belirlenirse 1 output çıkacak 1 çıkarsa biz bu köpek dicez o yüzden units 1 seçtik.
#eğer multiclass classification yapsaydık softmax activation kullanmamız gerekiyodu burda binary classification kullandığımız için sigmoid kullanıyoruz.

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'] ) #burda optimizere ve loss functiona bağlıcaz cnn i. adam optimizer seçiyoruz stochastic gradient descent yani erroru azaltmak için. loss olarak da cross entropy seçicez

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit( x = training_set , y = test_set , validation_data = test_set , epochs = 25) #we are not only training the CNN on the training set but also evaluating it at the same time on the test set bunun için de validation data kulanıyoruz.
#datasetimizin size ı çok büyük oldugu icin colabta çalışmıyor bu yüzden jupyterde run yapıcaz.

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg' , target_size = (64,64))  #burda image yüklüyoruz. training sette ne belirlediysek size ı burda da aynısını belirlememiz gerekiyor.
test_image = image.img_to_array(test_image) #predict method kullanıcaz ve her zamanki gibi predict method 2d array almak zorunda oldugu icin 2d array yapıcaz image i bunu da img_to_array ile sağlıyoruz.
test_image = np.expand_dims(test_image , axis = 0) #fake dimension or dimension corresponding to the batch. dimension of the batch is always first dimension bu yüzden axisi 0 seçicez yani biz batch e image eklicez ve batch in dimension ını first dimension
result = cnn.predict(test_image)
training_set.class.indices #1 köpeği mi belirtiyor kediyi mi bilmiyoruz bunun için bu kodu yazdık ve köpek 1 kedi 0 olacak.
if result[0][0] == 1:  #result contains also actually the result into a batch because it was called on a test_image that was into a batch so results also has this batch dimension. result[0] diyoruz çünkü sadece 1 batch var ve onun da indexi 0 and then inside the batch we are gonna get access to the first and only element of the batch which corresponds to the prediction of that same car_or_dog_1 image which corresponds 0
  prediction = 'dog'
else:
  prediction = 'cat'


In [ ]:
print(prediction)